<a href="https://colab.research.google.com/github/ABHAY1937/MedRecSys/blob/main/Medical_recomentation_LSTM_model_implimentaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import pickle


In [2]:
# Load CSV data into a DataFrame
df = pd.read_csv('/content/medical data.csv')
df.head()

,Name,DateOfBirth,Gender,Symptoms,Causes,Disease,Medicine
0,John Doe,15-05-1980,Male,"Fever, Cough",Viral Infection,Common Cold,"Ibuprofen, Rest"
1,Jane Smith,10-08-1992,Female,"Headache, Fatigue",Stress,Migraine,Sumatriptan
2,Michael Lee,20-02-1975,Male,Shortness of breath,Pollution,Asthma,Albuterol Inhaler
3,Emily Chen,03-11-1988,Female,"Nausea, Vomiting",Food Poisoning,Gastroenteritis,Oral Rehydration
4,Alex Wong,12-06-2001,Male,Sore Throat,Bacterial Infection,Strep Throat,Penicillin


In [3]:
# Drop rows with missing values in 'Symptoms' and 'Medicine', 'Disease' columns
df = df.dropna(subset=['Symptoms','Disease', 'Medicine'])


In [4]:
df.isnull().sum()

Name           1
DateOfBirth    1
Gender         1
Symptoms       0
Causes         1
Disease        0
Medicine       0
dtype: int64

In [5]:

# Preprocess text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Disease'].values)
input_sequences = tokenizer.texts_to_sequences(df['Disease'].values)


In [6]:
len(input_sequences)

242

In [7]:
with open('tokenizer.pickle','wb') as handle:
 pickle.dump(tokenizer,handle,protocol = pickle.HIGHEST_PROTOCOL) #saving the training model

In [8]:
max_sequence_len = max([len(seq) for seq in input_sequences])
max_sequence_len

3

In [9]:
vocab_size = len(tokenizer.word_index) + 1

In [10]:
# Pad sequences to a uniform length
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [11]:
# Encode target labels (Medicine)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['Medicine'].values)
# lab = label_encoder.fit_transform(df['Disease'].values)
# lab1 = label_encoder.fit_transform(df['Symptoms'].values)

In [12]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_sequences_padded, encoded_labels, test_size=0.30, random_state=42)

In [13]:
#Build an LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_len))
model.add(LSTM(128))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 100)            6800      
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 65)                8385      
                                                                 
Total params: 132,433
Trainable params: 132,433
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/200
6/6 [==============================] - 3s 122ms/step - loss: 4.1679 - accuracy: 0.1006 - val_loss: 4.1551 - val_accuracy: 0.3425
Epoch 2/200
6/6 [==============================] - 0s 13ms/step - loss: 4.1385 - accuracy: 0.3018 - val_loss: 4.1338 - val_accuracy: 0.2740
Epoch 3/200
6/6 [==============================] - 0s 14ms/step - loss: 4.1051 - accuracy: 0.3550 - val_loss: 4.1051 - val_accuracy: 0.2740
Epoch 4/200
6/6 [==============================] - 0s 13ms/step - loss: 4.0559 - accuracy: 0.2899 - val_loss: 4.0611 - val_accuracy: 0.1644
Epoch 5/200
6/6 [==============================] - 0s 15ms/step - loss: 3.9689 - accuracy: 0.2071 - val_loss: 3.9922 - val_accuracy: 0.1507
Epoch 6/200
6/6 [==============================] - 0s 14ms/step - loss: 3.8101 - accuracy: 0.1183 - val_loss: 3.8901 - val_accuracy: 0.1096
Epoch 7/200
6/6 [==============================] - 0s 14ms/step - loss: 3.5948 - accuracy: 0.1065 - val_loss: 3.8992 - val_accuracy: 0.1096
Epoch 8/200
6/6 [==

In [16]:
y_pred_probabilities = model.predict(X_test)

3/3 [==============================] - 0s 4ms/step


In [17]:
y_pred_classes = np.argmax(y_pred_probabilities, axis=1)

In [18]:
for patient_idx, predicted_class in enumerate(y_pred_classes):
    print(f"Name {patient_idx + 1}: Predicted Medicine Class {predicted_class}")

Name 1: Predicted Medicine Class 24
Name 2: Predicted Medicine Class 19
Name 3: Predicted Medicine Class 50
Name 4: Predicted Medicine Class 23
Name 5: Predicted Medicine Class 39
Name 6: Predicted Medicine Class 46
Name 7: Predicted Medicine Class 11
Name 8: Predicted Medicine Class 52
Name 9: Predicted Medicine Class 7
Name 10: Predicted Medicine Class 56
Name 11: Predicted Medicine Class 29
Name 12: Predicted Medicine Class 39
Name 13: Predicted Medicine Class 27
Name 14: Predicted Medicine Class 32
Name 15: Predicted Medicine Class 46
Name 16: Predicted Medicine Class 51
Name 17: Predicted Medicine Class 49
Name 18: Predicted Medicine Class 44
Name 19: Predicted Medicine Class 44
Name 20: Predicted Medicine Class 51
Name 21: Predicted Medicine Class 39
Name 22: Predicted Medicine Class 29
Name 23: Predicted Medicine Class 50
Name 24: Predicted Medicine Class 42
Name 25: Predicted Medicine Class 41
Name 26: Predicted Medicine Class 23
Name 27: Predicted Medicine Class 24
Name 28: Pr

In [19]:
y_pred=model.predict(X_test)

3/3 [==============================] - 0s 5ms/step


In [ ]:
#y_pred=np.where(y_pred > 0.6, 1,0)


In [20]:
model.save('medical.h5')

In [ ]:
# import pickle
# import tensorflow as tf

# # Load the tokenizer
# with open('/content/tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

# # Load the saved model
# model = tf.keras.models.load_model('medical.h5')

# # Get input from the user
# disease_name = input("Enter the disease name: ")

# # Tokenize the input
# input_sequence = tokenizer.texts_to_sequences([disease_name])
# padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen=max_sequence_len)

# # Make predictions using the model
# predictions = model.predict(padded_sequence)

# # Get the predicted medicine
# INDEX_TO_MEDICINE = df['Medicine']
# predicted_medicine_index = tf.argmax(predictions, axis=-1).numpy()[0]
# predicted_medicine = INDEX_TO_MEDICINE[predicted_medicine_index]  # Replace INDEX_TO_MEDICINE with your mapping

# print(f"recommended medicine for {disease_name}: {predicted_medicine}")


TESTING THE DATA

In [28]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
model = load_model('medical.h5')

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define input data
symptoms = input("Enter the symptoms :")
cause = input("Enter the cause")
disease_name = input("Enter the disease name: ")

# Preprocess input data
input_text = f"{symptoms} {disease_name} {cause}"
input_sequence = tokenizer.texts_to_sequences([input_text])
input_padded = pad_sequences(input_sequence, maxlen=max_sequence_len, padding='post')

# Make a prediction
predicted_label = model.predict(input_padded)
predicted_medicine_index = np.argmax(predicted_label)
predicted_medicine = label_encoder.inverse_transform([predicted_medicine_index])[0]

print(f"Predicted Medicine: {predicted_medicine}")

Enter the symptoms :headache
Enter the causestress
Enter the disease name: migraine
1/1 [==============================] - 0s 398ms/step
Predicted Medicine: Antibiotics, Rest
